# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd # импорт панды
from google.colab import drive # исмпорт функции поключения к гугл диску
drive.mount('/content/drive') # драйвер для подключения к гугл диску

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bd/data.csv')

In [ ]:
df.head(10) # осмотрим данные

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [ ]:
df.describe() # взглянем на стат данные 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.shape

(21525, 12)

In [ ]:
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [ ]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита
    

**Вывод**

- отсутствие строк, отклонение в трудовом стаже и ежемесячном доходе  19351 / 21525 строк
- min и max значения столбцов кол-во детей и дней стажа	- отрицательные либо слишком перспективные
- прыгающий регистр букв данных в столбце образование
- по столбцам ежемесячный доход и трудовой стаж значение после запятой
- минимальный возраст клиента 0 
- общий трудовой стаж в днях - лучше перевести в года

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
df.isnull().sum() # посчитатали кол-во пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

идентичное количество пропусков в днях работы и в ежемесячном доходе:возможно эти люди не работали или работали не официально

In [ ]:
# заменим пропуски на 0 
df['days_employed'] = df['days_employed'].fillna(0)
df['total_income'] = df['total_income'].fillna(0)

In [ ]:
# проверим кол-во пропусков после замены
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [ ]:
print ("{0:.0f}%".format((2174/21525)*100)) # процент пропусков

10%


**Вывод** 
пропуски заменили 0, так как нет смысла усреднять, тем более 10%, навряд ли это ошибка выгрузки данных,
видимо это те кто не работал и нет дохода, либо не работали оффициально.ю как мы предполагали

### Замена типа данных

In [ ]:
# конвертируем стож ы днях в года
df['days_employed'] = df['days_employed'] / 365

# заменим тип данных float на int и избавимся от отрицательных значений
df['days_employed'] = df['days_employed'].astype('int') 
df['days_employed'] = df['days_employed'].abs()
df['total_income'] = df['total_income'].astype('int')
df['total_income'] = df['total_income'].abs()


Просмотрим изменения

In [ ]:
df.head(5) # просмотрим шапку

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [ ]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,164.362602,43.293380,0.817236,0.972544,0.080883,1.505124e+05
std,1.381587,365.337729,12.574584,0.548138,1.420324,0.272661,1.098971e+05
min,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,1.000000,33.000000,1.000000,0.000000,0.000000,8.861200e+04
50%,0.000000,4.000000,42.000000,1.000000,0.000000,0.000000,1.355140e+05
75%,1.000000,13.000000,53.000000,1.000000,1.000000,0.000000,1.955430e+05
max,20.000000,1100.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Проверим соответсвие типов данных после изменений

In [ ]:
df.info() # типы данных соответсвуют заголовкам

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


* переименуем столбец стаж дней в года

In [ ]:
df = df.rename(columns={'days_employed' : 'year_employed'}) #переименовали стаж в года
df.head(0)


,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


* столбец переименован

**Вывод**

* заменен вещественный тип данных на целочисленный в стаже и доходе, методом .astype - преобразует (почти) любой тип в (почти) любой другой тип

* Переименовали столц стажа, дни в года


### Обработка дубликатов

In [ ]:
df.duplicated().sum() # определили количество дубликатов

54

In [ ]:
df = df.drop_duplicates().reset_index(drop = True) # удалили дубликаты, одновили индексацию

In [ ]:
df.duplicated().sum() # проверим наличие дубликатов после изменения

0

Поищем неявные дубликаты:

In [ ]:
df.dtypes 

children             int64
year_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

In [ ]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [ ]:
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

76 строк по 20 детей, предположим, что это опечатка
Погуглив, выяснилось следующее максимальное кол-во детей от одной женщины 69 (статья о Фёдоре Васильеве)
Максимальное кол-во родившихся детей за 1 раз - 10 персоналий, женщина из ЮАР Госиаме Ситхоле
Таких строк у нас 76, будем реалистами - заменим значение на 2

47 строк по минус -1 ребенку, это ни что иное как опечатка, заменим на 1

In [ ]:
df['children'] = df['children'].replace(20,2)

In [ ]:
df['children'] = df['children'].replace(-1,1)

Проверим данные

In [ ]:
df['children'].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Уже лучше

In [ ]:
df['dob_years'].unique() # среди уникальных значений видим 0, что странно


array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [ ]:
# выведем данные, кому меньше 16
df[df['dob_years'] < 16]

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,949,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,7,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,5,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,1090,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,3,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19784,0,0,0,среднее,1,женат / замужем,0,F,сотрудник,0,0,жилье
20413,0,928,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193,покупка своего жилья
20528,0,908,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788,недвижимость
21127,2,0,0,высшее,0,женат / замужем,0,M,компаньон,0,240702,строительство жилой недвижимости


In [ ]:
# сгруппируем данные по виду деятельности для дальнейшего усреднения возраста и последующей
# замены пустых значений на среднее по данной группировке
list_income_type = df['income_type'].unique().tolist()
list_income_type

['сотрудник',
 'пенсионер',
 'компаньон',
 'госслужащий',
 'безработный',
 'предприниматель',
 'студент',
 'в декрете']

In [ ]:
# напишем функцию обработки данных по категории занятости
def income(list):
    '''функция обработки данных по категории занятости'''
    
    # создаем df с фильтрами на фходные данные должность и возраст
    df_filter = df[(df['income_type'] == list) & (df['dob_years'] >= 16)]
    
    # выводим среднюю и изменив тип данных на целочисленный int
    df_filter_mean = df_filter['dob_years'].mean().astype('int')
    return df_filter_mean

In [ ]:
list_income_type_0 = [] # создали список 0
list_income_type_1 = [] # создали список 1
for i in list_income_type: # цикл пробегает по списку уникальных значений должностей
    list_income_type_0.append(i) # записываем данные должности в список
    list_income_type_1.append(income(i)) # записываем среднее значение возраста по должности из функции


In [ ]:
# создадим словарь, для поиска по ключу
slovar = dict(zip(list_income_type_0, list_income_type_1)) #создадим словарь из 2 списков
slovar # словарь из 2 списков

{'сотрудник': 40,
 'пенсионер': 59,
 'компаньон': 39,
 'госслужащий': 40,
 'безработный': 38,
 'предприниматель': 42,
 'студент': 22,
 'в декрете': 39}

In [ ]:
#создадим список индексов по фильту подлежащих корректировке
index_replace_dob_years = df[df['dob_years'] < 16].index.tolist()

In [ ]:
import time
from tqdm import tqdm

#напишем цикл замены некорректных значений 
for i in index_replace_dob_years:  # пробегаем по списку индексов подлеж-их корректировке
    if df.loc[i,'income_type'] in slovar: # есть ли наименование должность df в словаре ?
        df.loc[i,'dob_years'] = slovar[df.loc[i,'income_type']] # заменяем значение на корректное

In [ ]:
df[df['dob_years'] < 16] # проверили, есть ли значения меньше 16 лет

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [ ]:
df.dtypes

children             int64
year_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

In [ ]:

df['year_employed'].describe()

count    21471.000000
mean       164.775977
std        365.703757
min          0.000000
25%          1.000000
50%          4.000000
75%         13.000000
max       1100.000000
Name: year_employed, dtype: float64

In [ ]:
max_dob_years = df['dob_years'].max().astype('int') # максимальный возраст
max_dob_years

75

In [ ]:
mean_dob_years = df['dob_years'].mean().astype('int') # максимальный возраст
mean_dob_years

43

In [ ]:
max_year_employed = (max_dob_years-16).astype('int') # получили максимальный трудовой стаж, который может быть
max_year_employed

59

In [ ]:
df[df['year_employed'] > max_year_employed] # выведем наглядные данные о наших персоналиях

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
18,0,1096,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823,на покупку подержанного автомобиля
24,1,927,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547,операции с коммерческой недвижимостью
25,0,996,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112,покупка недвижимости
30,1,919,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21451,0,928,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439,сыграть свадьбу
21454,0,1058,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638,недвижимость
21455,0,992,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029,операции с недвижимостью
21464,0,1024,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем


In [ ]:
process = df[df['year_employed'] > max_year_employed]['year_employed'].count() # количчество строк подлежащих обработке
process

3445

In [ ]:
df.index[df['year_employed'] > max_year_employed] #индексы строк с некорректными данными

Int64Index([    4,    18,    24,    25,    30,    35,    50,    56,    71,
               78,
            ...
            21413, 21421, 21431, 21447, 21450, 21451, 21454, 21455, 21464,
            21467],
           dtype='int64', length=3445)

In [ ]:
fanatics_worker = df.index[df['year_employed'] > max_year_employed] # создали переменную со списком этих индексов

In [ ]:
# создали список с возрастом, для рассчета условно-приблизительного стажа
year_employed_fanatics_worker = [] # пустой список с годами персоналий
dob_years_fanatics_worker = [] # список трудовых фанатиков
for i in fanatics_worker:      # пробегаем по списку индексов фанатиков
    dob_years_fanatics_worker.append(df.dob_years[i]) # выстаскиваем возраст 
    year_employed_fanatics_worker.append(df.year_employed[i]) # выстаскиваем стаж
dob_years_fanatics_worker # список с возрастом персоналий
year_employed_fanatics_worker # список с годами стажа персоналий
df_work = pd.DataFrame(dob_years_fanatics_worker, fanatics_worker)
df_work

,0
4,53
18,53
24,57
25,67
30,62
...,...
21451,53
21454,62
21455,59
21464,59


In [ ]:
# получили условный cтаж сотрудников
df_work['real_work_year'] = df_work[0]-16
df_work

,0,real_work_year
4,53,37
18,53,37
24,57,41
25,67,51
30,62,46
...,...,...
21451,53,37
21454,62,46
21455,59,43
21464,59,43


In [ ]:
for i in fanatics_worker: # заменили некорректные данные стажа на условно приблизительные
    df.loc[i,'year_employed'] = df_work.loc[i,'real_work_year']

In [ ]:
df[df['year_employed'] > max_year_employed] # выведем наглядные данные о наших персоналиях

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [ ]:
x = []
for i in fanatics_worker:
    df.loc[i,'year_employed']
    x.append( df.loc[i,'year_employed'])
    
x_work = pd.DataFrame(x, fanatics_worker)
x_work

,0
4,37
18,37
24,41
25,51
30,46
...,...
21451,37
21454,46
21455,43
21464,43


In [ ]:
max_dob_years # максимальный возраст до корректирвоки из расчетов до корректировки

75

In [ ]:
max_year_employed # максимальный стаж из расчетов до корректировки

59

In [ ]:
df[df['year_employed'] > max_year_employed] # как видим, теперь нет данных превышающих максимальный стаж

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [ ]:
df[df['year_employed'] > max_year_employed].count()

children            0
year_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [ ]:
df.head(5) # вид уже лучше

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,37,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [ ]:
df['education'].unique() # просмотрим регистр кажого столбца

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [ ]:
df['education'] = df['education'].str.lower() # изменили регистр

In [ ]:
df['education'].unique() # проверили уникальные значения после корректировки

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [ ]:
df['education'] = df['education'].str.lower()

In [ ]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [ ]:
df['family_status'] = df['family_status'].str.lower()

In [ ]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [ ]:
df['income_type'].unique() # здесь все ок

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

**Вывод**

Выявили явные дубликаты при помощи duplicated().sum(), удалили с обновлением индексов df.drop_duplicates().reset_index(drop = True). 
Нашли неявные дубликаты в столбце кол-во детей, это -1 и 20, заменили на 1 и 2, подробное описание почему именно так, по тексты выше, в примечании.

Заменили некорректные значения в возрасте персоналий на среднее по виду деятельности

Привлек внимание стаж, нашли максимальный возраст персоналии, предположили, что трудовой стаж начался с 16 лет, а максимальный возраст персоналии 75 лет, нашли максимальный стаж (для использования проверки в дальнейшем) 59 лет.
Создали переменную со списком индексов подлежащим обработке, где максимальный стаж превышает 59 лет. При помощи цикла проставили условно-приблизительный стаж данным персоналиям и заменили данные.

Заменили значения возраста у сотрудников с возрастом 0 и удивительным стажем за сотни лет, таких строк выявили 101

При помощи unique(), проверили каждый столбец, выявлено отклонение в регистре, при помощи .str.lower() привели все к нижнему регистру.

# Лемматизация

In [ ]:
# Импортируем библиотеки
import json
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [ ]:
# Создадим функцию которая возвращает строку с ключевыми словами цели кредита
def lemma_pur(purpose):
    lemma = ' ' .join(m.lemmatize(purpose))
    return lemma

In [ ]:
# Добавляем столбец в наш df предварительно пропустив его через функцию
df['purpose_lem'] = df['purpose'].apply(lemma_pur)

In [ ]:
# Отобразим уникальные значения
df['purpose_lem'].unique()

array(['покупка   жилье \n', 'приобретение   автомобиль \n',
       'дополнительный   образование \n', 'сыграть   свадьба \n',
       'операция   с   жилье \n', 'образование \n',
       'на   проведение   свадьба \n', 'покупка   жилье   для   семья \n',
       'покупка   недвижимость \n',
       'покупка   коммерческий   недвижимость \n',
       'покупка   жилой   недвижимость \n',
       'строительство   собственный   недвижимость \n', 'недвижимость \n',
       'строительство   недвижимость \n',
       'на   покупка   подержать   автомобиль \n',
       'на   покупка   свой   автомобиль \n',
       'операция   с   коммерческий   недвижимость \n',
       'строительство   жилой   недвижимость \n', 'жилье \n',
       'операция   со   свой   недвижимость \n', 'автомобиль \n',
       'заниматься   образование \n',
       'сделка   с   подержанный   автомобиль \n',
       'получение   образование \n', 'свадьба \n',
       'получение   дополнительный   образование \n',
       'покупка   свой 

In [ ]:
# Создадим функцию (purpose_correct) для обработки категорий на основании результатов
def purpose_correct(list):
    if 'автомобиль' in list:
        return "автомобиль"
    if "образование" in list:
        return "образование"
    if "свадьба" in list:
        return "свадьба"
    if "недвижимость" in list or 'жилье' in list:
        return "недвижимость"
    if "строительство" in list:
        return "строительство"

In [ ]:
df['purpose_correct'] = df['purpose_lem'].apply(purpose_correct) #добавили новый столбец применив к нему функцию

In [ ]:
df.head(1) # вывели шапку, добавлено 2 новых столбца с леммами и корректировкой по леммам

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem,purpose_correct
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,недвижимость


In [ ]:
df.drop(['purpose', 'purpose_lem'], axis=1, inplace=True) # удалим столбец лемм и устаревший столбец категории

In [ ]:
df.head(1) # вывели шапку, видим - столбцы удалены

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_correct
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость


In [ ]:
unique_credit_purposes = df['purpose_correct'].unique().tolist()
unique_credit_purposes
# вывели катеории на приобретение кредита, их 4

['недвижимость', 'автомобиль', 'образование', 'свадьба']

In [ ]:
df.groupby('purpose_correct').count()

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income
purpose_correct,,,,,,,,,,,
автомобиль,4308,4308,4308,4308,4308,4308,4308,4308,4308,4308,4308
недвижимость,10814,10814,10814,10814,10814,10814,10814,10814,10814,10814,10814
образование,4014,4014,4014,4014,4014,4014,4014,4014,4014,4014,4014
свадьба,2335,2335,2335,2335,2335,2335,2335,2335,2335,2335,2335


**Вывод**

Выделены 4 леммы в значениях столбца 'purpose_correct с целями получения кредитного займа


### Категоризация данных

In [ ]:
df.head(5) # взглянем на таблицу

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_correct
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,37,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


In [ ]:
# зависимость образования на возврат кредита в срок
((df.groupby('education')['debt'].sum()/df.groupby('education')['debt'].count())*100).round(2)

education
высшее                  5.29
начальное              10.99
неоконченное высшее     9.14
среднее                 8.98
ученая степень          0.00
Name: debt, dtype: float64

In [ ]:
# соотношение количества берущих кредит, сгруппированных по образованию
# самыми закредитованными являются люди со средним образованием
((df.groupby('education')['debt'].count()/df.groupby('education')['debt'].count().sum())*100).round(2)

education
высшее                 24.46
начальное               1.31
неоконченное высшее     3.47
среднее                70.74
ученая степень          0.03
Name: debt, dtype: float64

In [ ]:
# зависимость гендера на возврат кредита в срок
((df.groupby('gender')['debt'].sum()/df.groupby('gender')['debt'].count())*100).round(2)

gender
F       7.01
M      10.26
XNA     0.00
Name: debt, dtype: float64

In [ ]:
((df.groupby('gender')['debt'].count()/df.groupby('gender')['debt'].count().sum())*100).round(2)

gender
F      66.08
M      33.91
XNA     0.00
Name: debt, dtype: float64

In [ ]:
# зависимость образования и гендера на возврат кредита в срок
((df.groupby(['education', 'gender'])['debt'].sum()/df.groupby(['education', 'gender'])['debt'].count())*100).round(2)

education            gender
высшее               F          4.72
                     M          6.49
начальное            F          9.82
                     M         12.61
неоконченное высшее  F          8.91
                     M          9.54
                     XNA        0.00
среднее              F          7.68
                     M         11.50
ученая степень       F          0.00
                     M          0.00
Name: debt, dtype: float64

In [ ]:
# напишем функцию категоризации по уровню дохода
def income_level(money):
    '''функция категоризации уровня дохода'''
    if money >= 250000:
        return "ультра высокий уровень : > 250"
    if 100000 <= money < 250000:
        return "высокий уровень : 100-250"
    if 50000 <= money < 100000:
        return "средний уровень дохода : 50-100"
    if money < 50000:
        return "низкий уровень дохода : < 50"

In [ ]:
# Добавим новый столбец, входными параметрами которого будет уровень дохода прошедший через функцию income_level
df['money_category'] = df['total_income'].apply(income_level) 

In [ ]:
df.head(5) # проверим резуьтат

,children,year_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_correct,money_category
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,ультра высокий уровень : > 250
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,высокий уровень : 100-250
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,высокий уровень : 100-250
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,ультра высокий уровень : > 250
4,0,37,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,высокий уровень : 100-250


**Вывод**

Видим, что % невозврата у мужчины выше, хотя мужчины берут кредиты реже женжин в 2 раза, женщины более ответсвенно подходят к обязательствам погашения.

Стоит отметить - самый высокий процент невозврата у людей с начальным образованием, но их колличество мало и составляет лишь 1.31% от общего числа заемщиков, в то время как, самыми закредитованными являются люди со средним образованием, их доля составляет 70.74%

Добавили новый столбец, с делением на категории доходности, это нам посволит найти ответы на ключевые вовпросы

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# оценим зависимость наличия детей на возврат кредита в срок
((df.groupby('children')['debt'].sum()/df.groupby('children')['debt'].count())*100).round(2)

children
0    7.54
1    9.16
2    9.49
3    8.18
4    9.76
5    0.00
Name: debt, dtype: float64

In [ ]:
children_pivot = df.pivot_table(index=['children'], columns=['debt'], values='education', aggfunc='count', fill_value=0)
children_pivot['ratio'] = (children_pivot[1] / children_pivot[0] * 100).round(2)
children_pivot

debt,0,1,ratio
children,,,
0,13044,1063,8.15
1,4411,445,10.09
2,1926,202,10.49
3,303,27,8.91
4,37,4,10.81
5,9,0,0.00


**Вывод**

Связь наличия детей на выплату кредита - ЕСТЬ, но в ощем обьеме - чаще всего берут кредиты: люди без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
                                                                                                 # знаков после запятой
((df.groupby('family_status')['debt'].sum()/df.groupby('family_status')['debt'].count())*100).round(2)

family_status
в разводе                7.11
вдовец / вдова           6.57
гражданский брак         9.32
женат / замужем          7.54
не женат / не замужем    9.75
Name: debt, dtype: float64

In [ ]:
family_pivot = df.pivot_table(index=['family_status'], columns=['debt'], values='education', aggfunc='count', fill_value=0)
family_pivot['ratio'] = (family_pivot[1] / family_pivot[0] * 100).round(2)
family_pivot

debt,0,1,ratio
family_status,,,
в разводе,1110,85,7.66
вдовец / вдова,896,63,7.03
гражданский брак,3775,388,10.28
женат / замужем,11413,931,8.16
не женат / не замужем,2536,274,10.80


**Вывод**

Самые высокие показатели у одиноких.

Зависимость - ЕСТЬ

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
((df.groupby(['money_category'])['debt'].sum()/df.groupby(['money_category'])['debt'].count())*100).round(2)

money_category
высокий уровень : 100-250          8.47
низкий уровень дохода : < 50       7.74
средний уровень дохода : 50-100    8.09
ультра высокий уровень : > 250     6.90
Name: debt, dtype: float64

In [ ]:
money_category_pivot = df.pivot_table(index=['money_category'], columns=['debt'], values='education', aggfunc='count')
money_category_pivot['ratio'] = money_category_pivot[1] / money_category_pivot[0] * 100
money_category_pivot

debt,0,1,ratio
money_category,,,
высокий уровень : 100-250,11053,1023,9.255406
низкий уровень дохода : < 50,2299,193,8.394954
средний уровень дохода : 50-100,3760,331,8.803191
ультра высокий уровень : > 250,2618,194,7.410237


**Вывод**

ДА - есть зависимость.

У людей с ультра высокий уровень дохода, самый маленький процент невозврата

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
((df.groupby(['purpose_correct'])['debt'].sum()/df.groupby(['purpose_correct'])['debt'].count())*100).round(2)

purpose_correct
автомобиль      9.35
недвижимость    7.23
образование     9.22
свадьба         7.97
Name: debt, dtype: float64

In [ ]:
purpose_pivot = df.pivot_table(index=['purpose_correct'], columns=['debt'], values='education', aggfunc='count', fill_value=0)
purpose_pivot['ratio'] = (purpose_pivot[1] / purpose_pivot[0] * 100).round(2)
purpose_pivot

debt,0,1,ratio
purpose_correct,,,
автомобиль,3905,403,10.32
недвижимость,10032,782,7.80
образование,3644,370,10.15
свадьба,2149,186,8.66


**Вывод**

Здесь явно выделяются подгруппы (недвижимость и свадьба), с самым маленьким процентов невозврата, скорее всего это связано с высокой ответственностью персоналий. Да и эти две подгруппы чаще всего идут параллельно и связаны )

Что за (авто и образование), здесь можно долго выводить гепотизы и человеский фактор не исколючение. Купленный авто сломался, требует дополнительных вложений - как пример. Образование требует дополнительных врменных вложений, возможно кто-то жертвует работой. либо ожидания обучения не оправданы, а то еще и хуже - образование не получено в конце-концов.

## Шаг 4. Общий вывод

Зависимость наличия детей и возвратом кредита в срок. ЕСТЬ.

Зависимость между семейным положением и возвратом кредита в срок. ЕСТЬ

Зависимость между уровнем дохода и возвратом кредита в срокю ЕСТЬ

Влияние цели кредита на его возврат в срок. ЕСТЬ

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.